In [3]:
import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import mnist_cnn

import visdom
import numpy as np


vis = visdom.Visdom()

def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num + 1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )


batch_size=32
epoch = 30
gpu = 1


T=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data = datasets.MNIST('./data',train=True,transform=T,download=True)
test_data = datasets.MNIST('./data',train=False,transform=T,download=True)

train_loader = data.DataLoader(train_data, batch_size=batch_size,shuffle=True,num_workers=4)
test_loader = data.DataLoader(test_data, batch_size=1,shuffle=True,num_workers=4)

CNN = mnist_cnn.MNIST_CNN()
if gpu == 1:
    CNN = CNN.cuda()

optimizer = optim.SGD(CNN.parameters(),lr = 0.002, momentum=0.5)

loss_fn = nn.CrossEntropyLoss()

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.9)

loss_plot = vis.line(Y=np.random.rand(1))


train_len = len(train_loader)
print(train_len)


for ep in range(epoch):
    lr_scheduler.step()
    for i, data in enumerate(train_loader):
        imgs, labels = data
        imgs, labels = Variable(imgs.cuda()), Variable(labels.cuda())

        optimizer.zero_grad()
        out, _ = CNN(imgs)

        loss = loss_fn(out,labels)
        loss.backward()
        optimizer.step()
        plt_tracker(loss_plot, (float(loss.data.cpu())), ((ep *1875) + i))

1875


In [9]:
torch.save(CNN,'./MNIST_CNN.pth')

In [10]:
#-------- test my network -------------

net = torch.load('./MNIST_CNN.pth')

total = 0
correct = 0

for data in test_loader:
    imgs, labels = data
    imgs, labels = Variable(imgs.cuda()), (labels.cuda())

    outputs, _ = net(imgs)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    
    
    if (int(predicted) == int(labels)):
        correct =correct +1 
    

print('Accuracy of the network : %f ' %(100* correct / total))




Accuracy of the network : 98.830000 
